## CCNSS 2018 module 2
# Tutorial 2 - Model fitting and model comparison

*Please execute the cell below to initialize the notebook environment*

In [1]:
from IPython.display import HTML
from IPython.display import display
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import scipy as sp                 # import scipy
import math                        # import basic math functions
import random                      # import basic random number generator functions

fig_w, fig_h = (6, 4)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
plt.style.use('ggplot')

In [2]:
# This code allows to call the function 'hide_toggle' that shows/hides solutions for each exercise

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Show/hide Solution below'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' '
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

---


## Objectives


In this notebook we'll look at *model fitting* using the Drift Diffusion Model (DDM) we developped in Tutorial 1. That is, we will attempt to recover the parameters that generated the data with the DDM. We will then have a look at *model comparison* and *model selection*.

Model fitting:

- Ordinary least squares (OLS)
- Likelihood, and Maximum Likelihood Estimation (MLE)
    - Grid search approximation
    - Gradient Descent optimisation

Model Comparison:

- Model Selection using:
    - Bayesian Information Criterion (BIC)
    - Akaike Information Criterion (AIC)

---


## Ordinary Least Squares (OLS) Regression

We have data pairs $(x_i,y_i)$ that we think are linearly related but we are not sure what the slope or intercept is that best characterizes this relationship. To find this, we fit the data with a linear model

\begin{align*} \hat{y}_i = \beta_1 x_i + \beta_0 \end{align*}

and estimate the best fitting parameters by minimizing the mean squared error (MSE). That is, we want to find the parameters $\beta_0$ and $\beta_1$ such that the line fits the data as best as possible. To do so, we will calculate the squared error (i.e. squared distance) between the line and the data-points, and choose the parameters that gives us the least error possible (i.e. the minimum error across all data points):

\begin{align*} \sum_i(y_i - \hat{y}_i)^2 = \sum_i (y_i-\beta_1 x_i+\beta_0)^2 \end{align*}

For the case of linear regression, there is actually an analytical solution:

\begin{align*} \beta_1 = \frac{cov(x,y)}{var(x)} \end{align*}

\begin{align*} \beta_0 = \bar{y} - \hat{\beta_1} x \end{align*}

but we will use a more general optimization library to start familiarizing ourselves with these tools.

***EXERCISE 1: OLS***

We will generate 100 data points with a linear relationship, and attempt to recover the parameters that generated the data.

**Suggestions**
* Set the seed to 0
* Generate N = 100 data pairs $(x_i,y_i)$ using a linear model with normally distributed noise $\epsilon = 1$ and your choice of slope $\beta_1=1$ and intercept $\beta_0=0.5$ parameters.
* Calculate the analytical estimate for the OLS regression
* Write a function that returns the mean square error (MSE) for any parameter values.
* Use an optimization library to numerically find the parameters that minimize the MSE and compare these to the true parameters of the generative model (hint: you may want to use the 'scipy.optimize.minimize' function.
* Plot the data, as well as the anlytical and numerical OLS estimate


In [3]:
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

from scipy.optimize import minimize

random.seed(0)

# Generate the data
beta1 = 1
beta0 = 0.5
sigma = 1
xdata = np.random.randn(100)
eps   = sigma * np.random.randn(100)
ydata = beta1*xdata + beta0 + eps

# Compute the analytical solutions
beta1_est = np.cov(xdata,ydata)[0,1] / np.var(xdata)
beta0_est = np.mean(ydata) - beta1_est*np.mean(xdata)
print('Analytical B0: ' + str(beta0_est), ', B1: ' + str(beta1_est))

# Function that returns the MSE
def get_MSE(params):   
    y_hat = params[1]*xdata + params[0]    
    return np.sum((ydata-y_hat)**2)

# Optimization (recovering the parameters)
initial_guess = [2,1]
res = minimize(get_MSE, initial_guess)
res.x
print('Optimization B0: ' + str(res.x[0]), ', B1: ' + str(res.x[1]))

plt.plot(xdata, ydata, 'o')
xs = np.array([xdata.min(), xdata.max()])
plt.plot(xs, beta0_est + beta1_est*xs,'-.b',LineWidth=2,label='Analytical Solution')
plt.plot(xs, res.x[0] + res.x[1]*xs,'--r',LineWidth=2,label='Optimization Solution')
plt.ylabel('Y');
plt.xlabel('X');

***Expected Output***

![](./figures/expected_ex1.png)

## Maximum Likelihood Estimation (MLE)

We can also fit the model to the data using Maximum Likelihood Estimation methods. To do this, we take the following generative model for the data:

\begin{align*} \hat{y}_i = \beta_1 x_i + \beta_0 + \epsilon \end{align*}

where $\epsilon\sim \mathcal{N}(\mu=0,\sigma^2)$ is a normally distributed random variable with mean 0 and variance $\sigma^2$ and $y\sim \mathcal{N}(\mu=(\beta_1 x+\beta_0),\sigma^2)$.

The probability distribution of $y$ given $x$ is then given by: 

\begin{eqnarray}
\\
p(y_i|x_i,\beta_1,\beta_0) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-(y_i-(\beta_1 x_i+\beta_0))^2/(2\sigma^2)\right]
\end{eqnarray}

For a pair $(x_i,y_i)$, the log likelihood of observation $y_i$ is 
\begin{eqnarray}
\log p(y_i|x_i,a,b)
= \log \left[\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-(y_i-(\beta_1 x_i+\beta_0))^2/(2\sigma^2)\right]\right] \\
= \log \left[\frac{1}{\sqrt{2\pi\sigma^2}}\right] -(y_i-(\beta_1 x_i+\beta_0))^2/(2\sigma^2)
\end{eqnarray}

That is: When $\epsilon$ is normally distributed, maximizing the total log likelihood of the data is equivalent to minimizing the mean squared error.


***EXERCISE 2: MLE - Grid-Search***

Using the generated 100 data points from EXERCISE 1, calculate the Likelihood of the data given the parameters. That is:

\begin{align*} p\left(Data_{y_i} \mid \beta_1,\beta_0 \right) \end{align*}

We will then extract the maximum of the log likelihood (or inversely, the minimum of the negative log likelihood). That is, the parameters that maximizes the chance of observing the data:

\begin{align*} p\left(\beta_1,\beta_0 \mid Data \right) = \arg \max \sum_{y_i} \log p\left(Data_{y_i} \mid \beta_1,\beta_0 \right)\end{align*}

***Suggestions***

* Write a function that returns the total negative of the log likelihood for any parameter values.
* Search (loop) through parameter values ranging from -4 to 4 in increments of 0.01, calculate and store the negative log likelihood for each parameter value of the loop.
* Print the parameters found for $\beta_0$, and $\beta_1$ to the true parameters.
* Print the error between the true parameters and those found using optimization, and time the function took to compute the likelihood for all pairs of parameter values.
* Plot the log likelihood as a function of parameter values ($\beta_0$ and $\beta_1$) using a heatmap, and mark the value with lowest negative log likelihood (hint: you may want to use the function `imshow`, or `contourf`)
    * Alternatively you could try to make fancier 3D-plots using : `plot_surface`, or `plot_wireframe`)

In [4]:
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

from mpl_toolkits.mplot3d import Axes3D
import time

# Function that returns the negative ll (nll)
def get_nll(params):
    y_hat = params[0]*xdata + params[1]
    
    # calculate the likelihood
    likelihood = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(ydata-y_hat)**2/(2*sigma**2))
    
    return -np.sum(np.log(likelihood+1e-100)) # keep l > 0 so can take log

step_x0 = 0.01
n_x0=len(np.arange(-4,4,step_x0))
beta1_all = np.tile(np.arange(-4,4,step_x0),n_x0)
beta0_all = np.repeat(beta1_all,n_x0)

x = y = np.arange(-4, 4, step_x0)
X, Y = np.meshgrid(x, y) 

t_start  = time.time()

ll_list = list()
for beta1_all_, beta0_all_ in zip(beta1_all,beta0_all):
    ll_list.append(-get_nll([beta1_all_, beta0_all_]))

t_end=time.time()

ll_list = np.array(ll_list)
extent = [np.min(beta1_all),np.max(beta1_all),np.min(beta0_all),np.max(beta0_all)]

ind = np.unravel_index(np.argmax(ll_list.reshape((n_x0,n_x0)), axis=None), ll_list.reshape((n_x0,n_x0)).shape)
print('Estimated B0: '+ str(Y[ind]) + ', estimated B1: ' +str(X[ind]) + ', estimated nLL: ' +str(ll_list.reshape((n_x0,n_x0))[ind]) )
print('Rounded Error B0: ' + str(round(Y[ind]-beta0,2)) + ' , rounded error B1: ' + str(round(X[ind]-beta1,2)) + ' , Time taken : ' + str(round(t_end-t_start,2)) + ' seconds')

# Plot using imshow()
plt.figure()
plt.title('Likelihood plot using: `imshow`')
plt.imshow(ll_list.reshape((n_x0,n_x0)), origin='lower',
            cmap='hot', extent=extent, aspect='auto')
plt.axvline(beta1,linestyle='-.',color='k')
plt.axhline(beta0,linestyle='-.',color='k')
plt.plot(beta1,beta0,'ok')
plt.xlabel(r'$\beta_1$')
plt.ylabel(r'$\beta_0$')
plt.colorbar(label='Log Likelihood')

# Plot using contourf()
plt.figure()
plt.title('Likelihood plot using: `contourf`')     
plt.contourf(X, Y, ll_list.reshape((n_x0,n_x0)), 40, cmap=plt.cm.bone)
plt.axvline(beta1,linestyle='-.',color='k')
plt.axhline(beta0,linestyle='-.',color='k')
plt.plot(beta1,beta0,'ok')
plt.xlabel(r'$\beta_1$')
plt.ylabel(r'$\beta_0$')
plt.colorbar(label='Log Likelihood')

***Expected output***

![](./figures/expected_ex2_1.png)

![](./figures/expected_ex2_2.png)

In [5]:
# Optional question: use plot_surface() and wire_frame()
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### --- Solution: OPTIONAL QUESTION 

# Plot using plot_surface()
fig = plt.figure()
plt.title('Likelihood plot using: `plot_surface`')
ax = Axes3D(fig)
surf = ax.plot_surface(X, Y, ll_list.reshape((n_x0,n_x0)), cmap=plt.cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel(r'$\beta_1$')
plt.ylabel(r'$\beta_0$')
ax.set_zlabel('Log Likelihood')

# Plot using wire_frame()
fig = plt.figure()
plt.title('Likelihood plot using: `contourf`')
ax = Axes3D(fig)
ax.plot_wireframe(X, Y, ll_list.reshape((n_x0,n_x0)), color='blue', rstride=20, cstride=20)
plt.xlabel(r'$\beta_1$')
plt.ylabel(r'$\beta_0$')
ax.set_zlabel('Log Likelihood')

***(Optional) Expected Output***

![](./figures/expected_ex2_3.png)

***EXERCISE 3: MLE - Optimization (Gradient Descent)***

Instead of looping through the combination of all the parameter values in a loop. We will be using gradient descent (i.e. 'optimization' methods), to extract the maximum of the log likelihood (or inversely, the minimum of the negative log likelihood). That is, the parameters that maximizes the chance of observing the data:

\begin{align*} p\left(\beta_1,\beta_0 \mid Data \right) = \arg \max \sum_{y_i} \log p\left(Data_{y_i} \mid \beta_1,\beta_0 \right)\end{align*}

Intuitively, you can think of gradient descent optimization as follows: 
The optimization algorithm calculates the slope (i.e. derivative) of a function at a given point. In our case, we want to maximize the likelihood (i.e. find the top of the log-likelihood 'hill'). That is, the optimization function will calculates the gradient (derivative) of the log-likelihood given the parameter values, and 'ascend' the likelihood function (or 'descend' the negative log-likelihood) until it finds a derivative of 0 (that is no slope --> it found a minima or maxima, up until convergence to some threshold)

***Suggestions***

* Use an optimization library to numerically find the parameters that minimize the negative log likelihood (or equivalently, maximize the log likelihood of the data given the model). Tip: Use the minimize function from the scipy.optimize module.
* Print the parameters found using the optimization function
* Print the error between the true parameters and those found using optimization, and time the function took to converge
* Compare the parameters found using optimization to those found with grid-search
* Compare the time it took to find the parameters with optimization vs. grid-search


In [6]:
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

# Optimization (find parameters that minimize nll)
initial_guess = [1,1]
t_start_optim = time.time()
res = minimize(get_nll, initial_guess)
t_end_optim = time.time()

print('Estimated B0: '+ str(res.x[1]) + ', estimated B1: ' +str(res.x[0]) + ', estimated nLL: ' +str(res.fun) )
print('Rounded Error B0: ' + str(round(res.x[1]-beta0,2)) + ' , rounded error B1: ' + str(round(res.x[0]-beta1,2)) + ' , Time taken : ' + str(round(t_end_optim-t_start_optim,8)) + ' seconds')
print(' ')
print('Grid-search took ' + str((t_end-t_start)/(t_end_optim-t_start_optim)) + ' times longer than using the `minimize` function')


***Expected Output***

![](./figures/expected_ex3_1.png)

# Application: Fitting the DDM

Now that we have looked at model fitting for a simple case, we can try to fit the DDM to the data from last class in order to find the best fitting mean, noise and boundary parameters.

We will test our ability to recover the parameters of the model on simulated data for which we set the parameters.

Once we are convinced that we can recover the parameters on simulated data.

***EXERCISE 4: Fit DDM to simulated data***
    
***Suggestions***

* Set your seed to 0
* Generate 5000 trials of simulated RT data using the constant bound DDM function ($\mu = 0.0015, \sigma = 0.05, B = 1$)
* Plot the simulated RT data using your plot function from yesterday

Tip: We will import a data simulation function `sim_DDM_constant`, a plotting function `plot_rt_distribution` and a function that computes the analytic DDM and returns the (RG) `analytic_ddm` from the module ddm. You can use this module or you can use your own based on the work from the last tutorial.

In [7]:
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

from src.ddm_tutorial1 import sim_DDM_constant, analytic_ddm, plot_rt_distribution

random.seed(0)

# Set parameters and initialize 
mu, sigma, B = 0.0015, 0.05, 1
n_trial      = 5000
rts_sim      = np.zeros(n_trial)
corrects_sim = np.zeros(n_trial)

# Simulate reaction time and performance data using the DDM constant bound model
for i_trial in range(n_trial):
    choice,correct,rt,dvTrace,tTrace = sim_DDM_constant(mu, sigma, B, seed=i_trial)
    rts_sim[i_trial] = rt
    corrects_sim[i_trial] = correct

# Plot the RT data, separating correct from incorrect trials
bins = np.linspace(0,2500,26)
plot_rt_distribution(rts_sim[corrects_sim==1], rts_sim[corrects_sim==0], bins)

***Expected output***

![](./figures/expected_ex4.png)

***EXERCISE 5: Compute likelihood from analytic DDM***

In this exercise we will calculate the likelihood given parameters for the Drift Diffusion Model.

***Suggestions***

* Implement the calculation of the DDM negative log-likelihood by completing the function below.
* Use the `analytic_ddm` function from the module `ddm_tutorial1` to calculate the log-likelihood for a correct trial where RT is $500ms$, $\mu=0.0015$ and $B=1$.
* What's the log-likelihood for an incorrect trial with otherwise identical parameters?
* What's the analytical log-likelihood of the decision-variable trajectory from the previous exercise?

In [8]:
# insert your code here

def get_nll_ddm(mu, sigma, B, rts, corrects):
    '''
    Determines the negative loglikelihood of the analytical DDM
    
    Parameters
    ----------
    parameter : array_like of float
        length 2: 1st entry is mu (drift rate), 2nd is B (boundary)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    sigma : float
        DDM standard deviation
    rts : array_like of floats
        reaction times for which the likelihood will be evaluated
    corrects: array_like of bools, same length as rts
        indicates for each rt if it was a correct trial
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''

hide_toggle(for_next=True)

In [ ]:
### Solution to function

def get_nll_ddm(mu, sigma, B, rts, corrects):
    '''
    Determines the negative loglikelihood of the analytical DDM
    
    Parameters
    ----------
    parameter : array_like of float
        length 2: 1st entry is mu (drift rate), 2nd is B (boundary)
        Note: we pack mu and B in one parameter because we want to
        make it compatible for later use with sp.optimize.minimize
    sigma : float
        DDM standard deviation
    rts : array_like of floats
        reaction times for which the likelihood will be evaluated
    corrects: array_like of bools, same length as rts
        indicates for each rt if it was a correct trial
        
    Returns
    -------
    nll : float
        negative log-likelihood
    '''
    
    # Make sure rts and corrects are numpy arrays
    rts = np.array(rts) # make sure rts is np.ndarray
    corrects = np.array(corrects)

    # Create a vector of values where we will evaluate the analytic_ddm
    teval = np.unique(rts)
    
    # Evaluate the analytic ddm
    dist_cor, dist_err = analytic_ddm(mu, sigma, B, teval, b_slope=0)

    # init log-likelihood (LL)
    ll = 0

    # For each RT determine the LL from the corresponding
    # correct or incorrect analytical RT proabability distribution
    # and sum LLs over occurrences of the RT
    
    # - correct trial RTs
    rts_cor = rts[corrects == True]
    rts_cor_unique, counts = np.unique(rts_cor, return_counts=True)
    for rt, count in zip(rts_cor_unique, counts):
        ll += count * np.log(dist_cor[np.where(teval == rt)[0][0]]) 

    # - incorrect trial RTs
    rts_err = rts[corrects == False]
    rts_err_unique, counts = np.unique(rts_err, return_counts=True)
    for rt, count in zip(rts_err_unique, counts):
        ll += count * np.log(dist_err[np.where(teval == rt)[0][0]])
        
    # return negative LL
    return -ll

hide_toggle(for_next=True)

In [ ]:
### Solution to exercise

# 500msec correct trial
print('Log-Likelihood using analytical DDM, correct trial, 500ms: ' +  str(-get_nll_ddm(1.5e-3, 0.05, 1, np.array([500]), np.array([True]))))

# 3. 500msec incorrect trial
print('Log-Likelihood using analytical DDM, incorrect trial, 500ms: ' + str(-get_nll_ddm(1.5e-3, 0.05, 1, np.array([500]), np.array([False]))))

# 4. LL of previous exercise's decision-variable trajectory
print('Log-Likelihood using analytical DDM, using decision-variable trajectory: ' + str(-get_nll_ddm(1.5e-3, 0.05, 1, rts_sim, corrects_sim)))

***Expected Output***

![](./figures/expected_ex5.png)

***EXERCISE 6: Fit DDM to simulated reaction time data***

Once you are able to evaluate your likelihood function at various parameter values, it's time to fit the simulated data. The goal here is to pass the negative log likelihood to an optimizer that will find the parameters to minimize the total negative log likelihood.

Note that optimizers tend to work better when parameters have the same order of magnitude. Also, the optimization function that we are going to use, `scipy.optimize.minimize`, requires that all parameters that are optimized over are packed into a vector and that this vector is the first argument of the objective function.

- Write a wrapper function that's exactly like `get_nll_ddm`, except that it takes as first argument the vector $(1000 \mu,B)$.

Remember that this will mean rescaling the parameters returned by the optimizer in future exercises!

In [9]:
# insert your code here

hide_toggle(for_next=True)

In [10]:
### Solution

# parameters = (1000mu, B)
get_nll_ddm_wrapper = lambda parameters, sigma, rts, corrects: get_nll_ddm(parameters[0]/1000., sigma, parameters[1], rts, corrects)

***Suggestions***

* Use the optimizer `minimize` on your negative log likelihood function to maximize the log likelihood of the simulated data. Again $\sigma$ will be fixed at 0.05. 
* Is the optimization succesful? If yes, you should see "message: 'Optimization terminated successfully.'" in the output. If not, consider using a bounded optimization (check out the bounds input to the function and use method 'SLSQP'). $mu$ and $B$ should be positive.
* Compare the simulated data with the fitted distribution. To do so, use the analytic_ddm function with the fitted parameter value.

In [11]:
# insert your code here

hide_toggle(for_next=True)

In [12]:
### Solution

from scipy.optimize import minimize

x0 = [1,2] # Initial guess
bounds = [(0.3,1.5),(0.5,3)] # Optimization bounds
sigma = 0.05 # always fixed

# The lambda expression allows us the pass different datasets
res = minimize(get_nll_ddm_wrapper, 
               args=(sigma, rts_sim, corrects_sim),
               method='SLSQP', x0=x0, bounds=bounds)
ll = -res.fun # get log likelihood

print(res.message)
print('Loglikelihood is :' + str(round(ll,2)))

# Plotting the simulated data
bins = np.linspace(0,2500,51)
plot_rt_distribution(rts_sim[corrects_sim==1], rts_sim[corrects_sim==0], bins)

# Plotting the analytical results
teval = np.arange(0,bins[-1],1)[1:]
mu = res.x[0]/1000 # scaling
b = res.x[1]
# sigma is as before
dist_cor, dist_err = analytic_ddm(mu, sigma, b, teval)

plt.plot(teval, dist_cor*(bins[1]-bins[0]), color = 'blue')
plt.plot(teval,-dist_err*(bins[1]-bins[0]), color = 'red')

NameError: name 'rts_sim' is not defined

***Expected output***

![](./figures/expected_ex6_1.png)

![](./figures/expected_ex6_2.png)

## Model Comparison

We will use the $BIC$ (Bayesian Information Criterion) to compare models:

\begin{align*} -2 \log p(M|y) \approx -2\ln(L) + k\ln(n) \equiv BIC \end{align*}

where 
                            * $M$ is the model under consideration, 
                            * $L$ the likelihood for model $M$,
                            * $y$ the observed data, 
                            * $k$ the number of free parameters, 
                            * $n$ the number of data points (observations)

and the approximation holds for large $n$.

The $BIC$ penalizes more complex models with more parameters. Specifically, in our context, the BIC penalizes the non-decision time model for its extra parameter.

Note that a lower BIC is better and in general a difference of BIC 10 or more is good evidence for the model with the lower BIC.

***EXERCISE 7 : Calculating the BIC***

We will compute the BIC for three models (one full model), one where we fix the $B=0.5$ to a constant, and one where $mu=0$ is fixed (you can think of this as a Null model)

***Suggestions***

* Fit the models with Compare the BIC for the three models (you can constrain parameter values by restricting the bounds of parameters to be optimized)
* Which model has the smaller BIC (the smaller the better)?
* Plot the BICs for each model

In [13]:
# insert your code here

hide_toggle(for_next=True)

In [ ]:
### Solution

N_data = len(rts_sim)
bic=list();

# Fit data to full model
res_full = minimize(get_nll_ddm_wrapper, 
               args=(sigma, rts_sim, corrects_sim),
               method='SLSQP', x0=x0, bounds=[(0.3,1.5),(0.5,3)])
ll_full = -res_full.fun # get log likelihood
bic.append(-2*ll_full+2*np.log(N_data))

# Fit data to restricted B model
res_B = minimize(get_nll_ddm_wrapper, 
               args=(sigma, rts_sim, corrects_sim),
               method='SLSQP', x0=x0, bounds=[(0.3,1.5),(0.5,0.5)])
ll_B = -res_B.fun # get log likelihood
bic.append(-2*ll_B+1*np.log(N_data))

# Fit data to restricted mu model
res_Null = minimize(get_nll_ddm_wrapper, 
               args=(sigma, rts_sim, corrects_sim),
               method='SLSQP', x0=x0, bounds=[(0,0),(0.5,3)])
ll_Null = -res_Null.fun # get log likelihood
bic.append(-2*ll_Null+1*np.log(N_data))

print('BIC for models: ')
print('Full Model BIC: ' + str(-2*ll_full+2*np.log(N_data)) )
print('Restricted B Model BIC: ' + str(-2*ll_B+1*np.log(N_data)) )
print('Restricted Mu Model BIC: ' + str(-2*ll_Null+1*np.log(N_data)) )

plt.figure()
plt.bar(range(3),bic-min(bic))
plt.xticks(range(3), ('Full', 'Restricted B', 'Restricted Mu'))
plt.ylabel('BIC difference vs. winning model')

***Expected output***

![](./figures/expected_ex7_1.png)

![](./figures/expected_ex7_2.png)